In [0]:
import sys
sys.path.insert(0, '../lib')

import utils
import ingestors

config = {
    "catalog": dbutils.widgets.get("catalog"),
    "schema": dbutils.widgets.get("schema"),
    "account": dbutils.widgets.get("account"),
    "tablename": dbutils.widgets.get("tablename"),
    "primary_key": dbutils.widgets.get("primary_key"),
    "id_field_old": dbutils.widgets.get("id_field_old"),
    "timestamp_field": dbutils.widgets.get("timestamp_field")
}

In [0]:
remove_checkpoint = False

if not utils.table_exists(spark, config["catalog"], config["schema"], config["tablename"]):

    print("Criando a tabela", config["tablename"])
    path = f"{config['tablename']}.sql"
    with open(path, "r") as open_file:
        query = open_file.read()
    (spark.sql(query)
          .write
          .format("delta")
          .mode("overwrite")
          .option("overwriteSchema", "true")
          .saveAsTable(f"{config['catalog']}.{config['schema']}.{config['tablename']}"))
    
    remove_checkpoint = True

else:
    print("tabela ja existente")

In [0]:
print("Iniciando CDF...")

ingest = ingestors.SilverIngestor(spark=spark,
                               config=config)

In [0]:
if remove_checkpoint:
    dbutils.fs.rm(ingest.checkpoint_location, True)

In [0]:
stream = ingest.run()